In [28]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [13]:
response_schemas = [
    ResponseSchema(name="Resume Text", description="Give these details <Name, Email, Phone_Number,Designation,Companies, Total_Year_experience, Skills,\
                    Certifications, College_Degree, Publications, Social_Links> from the given resume text.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [15]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="You are an excellent resume parser. Give necessary details from resume text.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [18]:
import PyPDF2
import docx

In [21]:
def convert_files_to_text2(fileObj,isPdf=True):
    text = ''
    if isPdf:
        reader = PyPDF2.PdfReader(fileObj)
        for page in reader.pages:
            text += page.extract_text()
    else:
        doc = docx.Document(fileObj)
        for paragraph in doc.paragraphs:
            text += paragraph.text + '\n'
    return text


def convert_pdf_to_text(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

def convert_docx_to_text(file_path):
    doc = docx.Document(file_path)
    text = ''
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'
    return text

In [22]:
import re
resume_text = convert_files_to_text2(r"C:\Users\koush\Documents\New Resumes\CKMourya_Resume.pdf")
def clean_resume_text(resume_text):
    # Convert to lowercase
    cleaned_text = resume_text.lower()
    
    # Remove non-alphanumeric characters
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)
    
    # Remove extra whitespaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    return cleaned_text


question= clean_resume_text(resume_text)


In [47]:
model = OpenAI(temperature=0)
_input = prompt.format_prompt(question=question)
output = model(_input.to_string())

In [44]:
print(output)

 tovisualize theclusters

```json
{
	"Resume Text": "Chinta Krishna Mourya, mourya.achinta19@gmail.com, 91-7-793981667, Data Science Intern, Ineuron, 1 year, Machine Learning, Natural Language Processing, Python, Neural Network, Statistical Modeling, SQL, Flask, Predictive Modeling, Statistics, Data Wrangling, Data Visualization, Git, Udemy Machine Learning, AZPython in Data Science, Ineuron Fullstack Data Science Bootcamp, English (Fluent), Telugu (Native), Hindi (Beginner)"
}
```


In [26]:
output_parser.parse(output)

OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)